<a href="https://colab.research.google.com/github/skauntey/movie-sentiment-analysis/blob/tf_tokenizer/Categoricalentropy_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir ~/.kaggle
!echo '{"username":"kaunteyshah","key":"e2678a9d0e44e057000b9daf059111b5"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os, shutil
import kaggle
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline  

#### 1. Downloading Kaggle files

In [ ]:
kaggle.api.authenticate()
kaggle.api.competition_download_files('sentiment-analysis-on-movie-reviews', path= str(os.getcwd())+"/dataset/", force = True)

#### 2. Unzipping Kaggle files

In [ ]:
import zipfile
ziped_file = r'dataset/sentiment-analysis-on-movie-reviews.zip'
folder_path = os.path.join(str(os.getcwd()), 'dataset/')
for files in os.listdir(folder_path):
  path = os.path.join(os.getcwd()+'/'+'dataset/',files)
  if path.split('.')[-1] != "zip":
    try:
      shutil.rmtree(path)
    except:
      raise
with zipfile.ZipFile(ziped_file) as zip_file:
  for member in zip_file.namelist():
      if member.split('.')[-1] == "zip":
          fdir = member.split('.')[0]
          zip_file.extract(member, path= os.path.join('dataset/'+fdir))
          # extracting individual train and test files in their respective folders
          zippedfile_name = os.listdir(os.path.join(str(os.getcwd()+'/dataset/'+fdir+'/')))
          zippedfile = os.path.join(str(os.getcwd())+'/dataset/'+ fdir, zippedfile_name[0])
          with zipfile.ZipFile(zippedfile, mode='r') as tsv_zip:
              tsv_zip.extractall(path = os.path.join('dataset/'+fdir+'/'))
                
          os.remove(zippedfile)
      else:
        continue
            
os.remove(ziped_file)


#### 3. Testing files

In [ ]:
tsvfile = 'dataset/train/train.tsv'
# read the data
tsv_read = pd.read_csv(tsvfile, sep="\t")

original_dataset = pd.DataFrame({ 
    'DATA_COLUMN': tsv_read['Phrase'], 
    'LABEL_COLUMN': tsv_read['Sentiment']})

pdf_data = original_dataset['DATA_COLUMN']
pdf_labels = original_dataset['LABEL_COLUMN']

In [ ]:
sorted_data = original_dataset['LABEL_COLUMN'].value_counts().sort_values(ascending=False)
sorted_data.plot(kind = 'bar')

In [ ]:
import re
original_dataset = original_dataset.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]


def clean_text(text):
  """
    text: a string
    return: modified initial string
  """
  text = text.lower() # lowercase text
  text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
  text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
  text = text.replace('x', '')
  text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
  return text


original_dataset['DATA_COLUMN'] = original_dataset['DATA_COLUMN'].apply(clean_text)
original_dataset.head(10)


In [ ]:
original_dataset.head()
pdf_dataset = original_dataset[original_dataset['DATA_COLUMN'].astype(bool)]
pdf_dataset = pdf_dataset.reset_index(drop=True)
pdf_dataset.head()

In [ ]:
train_split = 0.8
val_split = 0.2
test_split = 0

training_mark = int(abs(len(pdf_dataset) * train_split))
# validation_mark = np.add(training_mark, int(abs(len(pdf)) * val_split))

print(training_mark)
# print(validation_mark)

# Based on Data and Labels across the whole dataset
pdf_dataset_data = pdf_dataset['DATA_COLUMN'].values
pdf_dataset_labels = pdf_dataset['LABEL_COLUMN'].values

# One Hot Vector of the label dataset

pdf_dataset_labels_onehot = pd.get_dummies(pdf_dataset_labels).values
print('Shape of label tensor:', pdf_dataset_labels_onehot.shape)

# Train/Test split without One_hot
training_set = pdf_dataset[1:training_mark]
validation_set = pdf_dataset[training_mark:]


train_data = training_set['DATA_COLUMN']
train_labels = training_set['LABEL_COLUMN']

val_data = validation_set['DATA_COLUMN']
val_labels = validation_set['LABEL_COLUMN']

print(val_data.head())
print(val_labels.head())

print(train_data.head())
print(train_labels.head())

print(pdf_dataset_data[:10])
print(type(train_data))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, SpatialDropout1D, BatchNormalization, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.compat.v1.keras.layers import CuDNNLSTM, CuDNNGRU

In [ ]:
"""The sentiment labels are:

0 - negative
1 - somewhat negative
2 - neutral
3 - somewhat positive
4 - positive"""

In [ ]:
"""def num_words (key_dataset):
  total_words = {}
  for x in key_dataset:
    for a in x.split():
      if a not in total_words.keys():
        total_words[a] = 1
      else:
        total_words[a] += 1
  return len(total_words.keys())

num_words = num_words(pdf_dataset_data)
num_words"""
#16245

In [ ]:
"""def max_len(key_dataset):
  _, pretrained_tokenizer = tokenizing_dataset(key_dataset)
  sequences = pretrained_tokenizer.texts_to_sequences(key_dataset)
  max_sentence = max([len(x) for x in sequences])
  return max_sentence

max_len = max_len(pdf_dataset_data)
max_len"""
#31

In [ ]:


def max_len(key_dataset):
  _, pretrained_tokenizer = tokenizing_dataset(key_dataset)
  sequences = pretrained_tokenizer.texts_to_sequences(key_dataset)
  max_sentence = max([len(x) for x in sequences])
  return max_sentence


def tokenizing_dataset(key_dataset):
  tokenizer = Tokenizer(num_words = 16245,oov_token='<00V>')
  tokenizer.fit_on_texts(key_dataset)
  word_index = tokenizer.word_index
  return word_index, tokenizer

def Sequencing_dataset(key_dataset):
  _, pretrained_tokenizer = tokenizing_dataset(key_dataset)
  sequences = pretrained_tokenizer.texts_to_sequences(key_dataset)
  padded_sentences = pad_sequences(sequences, maxlen= 31, truncating ='post', padding='post')
  return padded_sentences

pdf_padded_dataset = Sequencing_dataset(pdf_dataset_data)
print(type(pdf_padded_dataset))


In [ ]:
def decoding_sequences(original_dataset, sequences):
  word_index, _ = tokenizing_dataset(original_dataset)
  reversed_index = dict(map(reversed, word_index.items()))
  for letter in range(len(sequences)):
      reversed_index.get(letter) 
  reversed_sequence = [reversed_index.get(letter) for letter in sequences]
  return (' '.join(reversed_sequence))
    
print(decoding_sequences(pdf_dataset_data, train_sequences[0]))
print(train_data[:1][1])

In [ ]:
# One-hot split of the Labels

pdf_dataset_labels_onehot = pd.get_dummies(pdf_dataset_labels)
print('Shape of label tensor:', pdf_dataset_labels_onehot.shape)

train_labels_onehot =  pdf_dataset_labels_onehot[1:training_mark]
val_labels_onehot = pdf_dataset_labels_onehot[training_mark:]

print(train_labels_onehot.head(5))
print(val_labels_onehot.head(5))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(pdf_padded_dataset,pdf_dataset_labels_onehot, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
vocabulary_size = 16245
embedding_dim = 128
max_length = 31
trunc_type = 'post'
oov_tok = '<00V>'

model = tf.keras.Sequential([
                             
    tf.keras.layers.Embedding(vocabulary_size, embedding_dim, input_length = max_length),
    #tf.keras.layers.Flatten(),
    #tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.SpatialDropout1D(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,dropout=0.4, recurrent_dropout=0.4, return_sequences=True)),
    tf.compat.v1.keras.layers.CuDNNLSTM(32),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(5, activation ='sigmoid')
])


model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999), metrics=['accuracy'])
print(model.summary())

In [ ]:
num_epochs = 100
batch_size = 5000
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs = num_epochs, validation_split=0.1)

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)